# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
vacation_df = pd.read_csv("../../Instructions/starter_code/City_data.csv")
vacation_df.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,alofi,-19.0595,-169.9187,75.09,94.0,75.0,10.36,NU,1.665500e+09
1,upernavik,72.7868,-56.1549,20.10,68.0,100.0,4.43,GL,1.665500e+09
2,ushuaia,-54.8000,-68.3000,49.66,46.0,75.0,11.50,AR,1.665500e+09
3,georgetown,5.4112,100.3354,83.89,79.0,20.0,5.75,MY,1.665500e+09
4,rosarito,32.3333,-117.0333,66.47,81.0,36.0,4.38,MX,1.665500e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
gmaps.configure(api_key=g_key)

In [30]:
vacation_df = vacation_df.dropna()
humidity = vacation_df["Humidity"].astype(float)

locations = vacation_df[["Latitude", "Longitude"]].astype(float)

figure_1 = gmaps.figure(center = [0,0] ,zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100, point_radius = 5)

figure_1.add_layer(heat_layer)

figure_1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
filtered_vacation_df = vacation_df

filtered_vacation_df = filtered_vacation_df.loc[(filtered_vacation_df["Max Temp"] < 80) 
                                & (filtered_vacation_df["Max Temp"] > 70) \
                                & (filtered_vacation_df["Wind Speed"] < 10) \
                                & (filtered_vacation_df["Cloudiness"] == 0)].dropna()
                                
filtered_vacation_df.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
80,kapaa,22.0752,-159.3190,73.72,78.0,0.0,9.22,US,1.665500e+09
100,san rafael,-34.6177,-68.3301,71.94,14.0,0.0,1.07,AR,1.665500e+09
109,pascagoula,30.3658,-88.5561,71.33,64.0,0.0,5.75,US,1.665500e+09
146,diego de almagro,-26.3667,-70.0500,72.07,11.0,0.0,6.26,CL,1.665500e+09
178,angoche,-16.2325,39.9086,79.05,70.0,0.0,8.68,MZ,1.665500e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
hotel_df = filtered_vacation_df

params = {"radius": 5000,
              "types": "lodging",
              "key": g_key
}

for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    params["location"] = f"{latitude},{longitude}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=parameters).json()
    
  
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    

Retrieving Results for Index 80: kapaa.
Missing field/result... skipping.
------------
Retrieving Results for Index 100: san rafael.
Missing field/result... skipping.
------------
Retrieving Results for Index 109: pascagoula.
Missing field/result... skipping.
------------
Retrieving Results for Index 146: diego de almagro.
Missing field/result... skipping.
------------
Retrieving Results for Index 178: angoche.
Missing field/result... skipping.
------------
Retrieving Results for Index 215: salalah.
Missing field/result... skipping.
------------
Retrieving Results for Index 276: muros.
Missing field/result... skipping.
------------
Retrieving Results for Index 306: la reforma.
Missing field/result... skipping.
------------
Retrieving Results for Index 324: rania.
Missing field/result... skipping.
------------
Retrieving Results for Index 327: santiago del estero.
Missing field/result... skipping.
------------
Retrieving Results for Index 335: mersin.
Missing field/result... skipping.
-

In [45]:
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
80,kapaa,22.0752,-159.3190,73.72,78.0,0.0,9.22,US,1.665500e+09
100,san rafael,-34.6177,-68.3301,71.94,14.0,0.0,1.07,AR,1.665500e+09
109,pascagoula,30.3658,-88.5561,71.33,64.0,0.0,5.75,US,1.665500e+09
146,diego de almagro,-26.3667,-70.0500,72.07,11.0,0.0,6.26,CL,1.665500e+09
178,angoche,-16.2325,39.9086,79.05,70.0,0.0,8.68,MZ,1.665500e+09
215,salalah,17.0151,54.0924,79.11,68.0,0.0,3.94,OM,1.665500e+09
276,muros,42.7762,-9.0603,76.84,63.0,0.0,6.06,ES,1.665500e+09
306,la reforma,25.0833,-108.0500,79.74,54.0,0.0,4.03,MX,1.665500e+09
324,rania,29.5333,74.8333,77.83,67.0,0.0,7.38,IN,1.665500e+09
327,santiago del estero,-27.7951,-64.2615,73.90,20.0,0.0,4.85,AR,1.665500e+09


In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

KeyError: 'Hotel Name'

In [52]:
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color = 'rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info)

marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(maker_layer)

fig

NameError: name 'hotel_info' is not defined